In [ ]:
                
import warnings
warnings.filterwarnings("ignore")

import os
import wandb
from math import ceil
from transformers import (
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

# wandb_dir = os.path.join(os.getcwd(), 'wandb_runs')
# os.makedirs(wandb_dir, exist_ok=True)

# Import our custom modules
from config import Config
from metrics import create_compute_metrics
from data_utils import set_seeds, load_and_prepare_dataset, preprocess_dataset
from utils import clear_memory, create_directories, safe_training_check, save_model_safe
from train import setup_model, create_training_args, create_callbacks

def main():
    """Main training function"""
    
    # Load config
    config = Config()
    
    print("=" * 60)
    print("🚀 Nepali Grammar Error Correction Training")
    print("=" * 60)
    print(f"Model: {config.model_id}")
    print(f"LoRA: {config.use_lora}")
    print(f"Samples: {config.num_samples or 'Full dataset'}")
    print("=" * 60)
    
    # Setup
    set_seeds(config.seed)
    clear_memory()
    create_directories(config.output_dir)
    
    # Initialize wandb
    wandb.finish()
    wandb.init(
        project=config.wandb_project,
        config=vars(config),
        # dir=wandb_dir
    )
    run_id = wandb.run.id
    
    # Load data
    dataset = load_and_prepare_dataset(config)
    
    # Setup model
    model, tokenizer = setup_model(config)
    
    # Preprocess
    dataset_encoded = preprocess_dataset(dataset, tokenizer, config)
    
    # Create training args
    training_args = create_training_args(config, dataset_encoded, run_id)
    
    # Data collator
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        padding=True
    )
    
    # Create metrics
    compute_metrics = create_compute_metrics(tokenizer, config)
    
    # Create callbacks
    callbacks = create_callbacks(config, tokenizer, dataset)
    
    # Create trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset_encoded["train"],
        eval_dataset=dataset_encoded["valid"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=callbacks,
    )
    
        # Safety check
    if not safe_training_check(trainer):
        print("\n❌ Safety checks failed! Fix issues before training.")
        return
    
    # Train!
    print("\n" + "=" * 60)
    print("🏋️  Starting training...")
    print("=" * 60)
    
    try:
        if config.resume_from_checkpoint:
            print("continuing training from latest checkpoint.....")
            trainer.train(resume_from_checkpoint=True)
        else:
            trainer.train()
        print("\n✅ Training complete!")
    except Exception as e:
        print(f"\n❌ Training failed: {e}")
        wandb.finish()
        return
    
    # Save model
    best_model_path = f"{config.output_dir}/best_model"
    save_model_safe(model, tokenizer, best_model_path, use_lora=config.use_lora)
    
    print(f"\n🎉 All done! Model saved to {best_model_path}")
    wandb.finish()

if __name__ == "__main__":
    main()


🚀 Nepali Grammar Error Correction Training
Model: google/mt5-small
LoRA: True
Samples: 15
✅ Seeds set to 42
Memory cleared
Directories created in c:\Users\Lenovo\Desktop\Nepali_GEC\nepali_gec\outputs


wandb: Currently logged in as: lsumit008 (lsumit008-khwopa-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\logging\__init__.py", line 1153, in emit
    stream.write(msg + self.terminator)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode characters in position 187-191: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Lenovo\Desktop\Nepali_GEC\nepali_gec\myenv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    


📚 Loading dataset: sumitaryal/nepali_grammatical_error_correction
  Using 15 samples
  Train: 13 samples
  Valid: 2 samples

 Lodaing model: google/mt5-small
 Using LoRA + 8-bit quantization
trainable params: 1,376,256 || all params: 301,553,024 || trainable%: 0.4564

⚙️  Preprocessing dataset...


Map: 100%|██████████| 2/2 [00:00<00:00, 89.28 examples/s]

  ✅ Preprocessing complete

📊 Training plan:
  Steps per epoch: 1
  Total steps: 3
  Warmup steps: 0
Enabled metrics:bleu, chrf, gleu, correction_accuracy


...

Epoch,Training Loss,Validation Loss,Model Preparation Time,Bleu,Gleu,Chrf,Correction Accuracy
1,21.034600,9.093679,0.002500,0.000000,0.000000,0.000000,0.000000
2,24.705000,8.171597,0.002500,0.000000,0.000000,0.000000,0.000000
3,25.778200,8.344097,0.002500,0.000000,0.000000,0.000000,0.000000


🔍 Sample - Pred: '<extra_id_0>...' | Ref: 'फर्कने हो उतै?...' | Match: False
📊 Logged 5 sample predictions to W&B
🔍 Sample - Pred: '<extra_id_0>...' | Ref: 'फर्कने हो उतै?...' | Match: False
📊 Logged 5 sample predictions to W&B
🔍 Sample - Pred: '<extra_id_0>...' | Ref: 'फर्कने हो उतै?...' | Match: False
📊 Logged 5 sample predictions to W&B

✅ Training complete!

💾 Saving model to c:\Users\Lenovo\Desktop\Nepali_GEC\nepali_gec\outputs/best_model...


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


  ✅ LoRA adapter saved
  ✅ Tokenizer saved

🎉 All done! Model saved to c:\Users\Lenovo\Desktop\Nepali_GEC\nepali_gec\outputs/best_model


epoch,▁▅█
eval/bleu,▁▁▁▁
eval/chrf,▁▁▁▁
eval/correction_accuracy,▁▁▁▁
eval/gleu,▁▁▁▁
eval/loss,█▄▁▂
eval/model_preparation_time,▁▁▁▁
eval/runtime,▁▂█▅
eval/samples_per_second,█▇▁▄
eval/steps_per_second,█▇▁▄
+5,...
